In [251]:
import pandas as pd

In [252]:
import time

start_time = time.time()

In [253]:
def cabecera_resumen_mot(row, format):
    worksheet.write('A' + str(row), 'Categoría', format)
    worksheet.write('B' + str(row), 'Cadena', format)
    worksheet.write('C' + str(row), 'Si', format)
    worksheet.write('D' + str(row), 'Si pero el precio no corresponde', format)
    worksheet.write('E' + str(row), 'No ha caido la promoción', format)
    worksheet.write('F' + str(row), 'No puedo señalizar', format)
    worksheet.write('G' + str(row), 'No tengo suficiente producto', format)
    worksheet.write('H' + str(row), 'No he ejecutado', format)
    worksheet.write('I' + str(row), 'Ejecutado', format)
    worksheet.write('J' + str(row), 'NO Ejecutado', format)
    worksheet.write('K' + str(row), '% Ejecución', format)
    worksheet.write('L' + str(row), '% No ha caido la promoción', format)
    worksheet.write('M' + str(row), '% No puedo señalizar', format)
    worksheet.write('N' + str(row), '% No tengo suficiente producto', format)
    worksheet.write('O' + str(row), '% No he ejecutado', format)

In [254]:
def cabecera_resumen_gos(row, format):
    worksheet.write('B' + str(row), 'Detergentes', format)
    worksheet.write('C' + str(row), 'Suavizantes', format)
    worksheet.write('D' + str(row), 'Lavatrastes', format)
    worksheet.write('E' + str(row), 'Detergentes', format)
    worksheet.write('F' + str(row), 'Suavizantes', format)
    worksheet.write('G' + str(row), 'Lavatrastes', format)

In [255]:
stores_file = 'stores_clean.xlsx'

In [256]:
stores = pd.read_excel(stores_file, 0)

In [257]:
belleza = pd.read_excel('Plan Promocional Quimicos Mayo 2017.xlsx', 4, 2)

In [258]:
preguntas = belleza[['Categoría', 'NOMBRE DE LA INICIATIVA']]

In [259]:
iniciativa_belleza = pd.read_csv('lecturas_quimicos_17.csv', encoding='mbcs')

In [260]:
writter = pd.ExcelWriter('RepPromoQuimicos.xlsx')

In [261]:
workbook = writter.book

In [262]:
porcentaje = workbook.add_format({'num_format': '0%'})

In [263]:
iniciativas = iniciativa_belleza[
    ['Cadena', 'Región', 'Zona', 'Tienda ID', 'Nombre Tienda', 'Fecha Captura', 'Grupo Categorias',
     'Iniciativa', 'Apoyo', 'Respuesta Opc.Multiple/Texto Abierto']]

In [264]:
respuestas = iniciativas['Respuesta Opc.Multiple/Texto Abierto'].replace('1.Si\r\n2.No', 'Si')
respuestas = respuestas.replace('he ejecutado\r\n6.No', 'No he ejecutado')
respuestas = respuestas.replace('he ejecutado\n6.No', 'No he ejecutado')
respuestas = respuestas.replace('he ejecutado', 'No he ejecutado')
respuestas = respuestas.replace('no tengo suficiente producto\r\n3.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no tengo suficiente producto\n3.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no ha caido la promoción\r\n4.No', 'No ha caido la promoción')
respuestas = respuestas.replace('no ha caido la promoción\n4.No', 'No ha caido la promoción')
respuestas = respuestas.replace('1.Si\r\n2.Si', 'Si')
respuestas = respuestas.replace('1.Si\n2.Si', 'Si')
respuestas = respuestas.replace('1.Si\n2.No', 'Si')
respuestas = respuestas.replace('pero el precio no corresponde.  \r\n3.No', 'Si pero el precio no corresponde')
respuestas = respuestas.replace('pero el precio no corresponde.  \n3.No', 'Si pero el precio no corresponde')
respuestas = respuestas.replace('no ha caido la promoción\r\n5.No', 'No ha caido la promoción')
respuestas = respuestas.replace('no ha caido la promoción\n5.No', 'No ha caido la promoción')
respuestas = respuestas.replace('no tengo suficiente producto\r\n4.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no tengo suficiente producto\n4.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no puedo señalizar\r\n5.No', 'No puedo señalizar')
respuestas = respuestas.replace('no puedo señalizar\r\n6.No', 'No puedo señalizar')
respuestas = respuestas.replace('no puedo señalizar\n6.No', 'No puedo señalizar')

In [265]:
iniciativas = iniciativas.join(respuestas, rsuffix='_respuestas')

In [266]:
preguntas_id = iniciativas['Apoyo'].str.extract("(?P<PreguntaID>\w{5}\d{1,3})(?P<Pregunta>¿.+)", expand=False)

In [267]:
iniciativas = iniciativas.join(preguntas_id)

In [268]:
preguntas = preguntas.set_index('NOMBRE DE LA INICIATIVA')

In [269]:
iniciativas = iniciativas.join(preguntas, on='Pregunta')

In [270]:
stores = stores[['Stores ID', '# Sucursal Cliente', 'Cadena', 'Formato', 'Nombre Tienda', 'Estatus de Tienda', 'Canal']]

In [271]:
stores = stores.drop_duplicates()

In [272]:
iniciativas = iniciativas.join(stores.set_index('Stores ID'), on='Tienda ID', rsuffix='_stores')

In [273]:
columnas_iniciativas = iniciativas.columns

In [274]:
iniciativas = iniciativas.sort_values(by='Fecha Captura')

In [275]:
iniciativas = iniciativas.drop_duplicates(['Nombre Tienda', 'Pregunta'], keep='last').values

In [276]:
iniciativas = pd.DataFrame(iniciativas, columns=columnas_iniciativas)

In [277]:
data = iniciativas[['Canal', 'Categoría', 'Cadena', 'Región', 'Zona', 'Tienda ID', 'Nombre Tienda', 'Fecha Captura',
                    'Iniciativa', 'Pregunta', 'Respuesta Opc.Multiple/Texto Abierto_respuestas']]

In [278]:
order = data.groupby(['Canal', 'Categoría', 'Cadena', 'Pregunta', 'Respuesta Opc.Multiple/Texto Abierto_respuestas'],
                     as_index=False).count()

In [279]:
order = order[['Canal', 'Categoría', 'Cadena', 'Pregunta', 'Respuesta Opc.Multiple/Texto Abierto_respuestas', 'Zona']]

In [280]:
categorias = order.pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                               columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona', fill_value=0).reset_index(
    ['Canal', 'Categoría', 'Cadena', 'Pregunta'])

In [281]:
categorias = categorias.assign(Ejecutado=(categorias['Si'] + categorias['Si pero el precio no corresponde']))

In [282]:
categorias = categorias.assign(No_Ejecutado=(
    categorias['No ha caido la promoción'] + categorias['No he ejecutado'] + categorias['No puedo señalizar'] +
    categorias['No tengo suficiente producto']))

In [283]:
categorias = categorias.assign(
    Porcentaje_Ejecución=(categorias['Ejecutado'] / (categorias['Ejecutado'] + categorias['No_Ejecutado'])))

In [284]:
categorias = categorias.rename(columns={'No_Ejecutado': 'No Ejecutado', 'Porcentaje_Ejecución': '% Ejecución'})

In [285]:
categorias = categorias[['Canal', 'Categoría', 'Cadena', 'Pregunta', 'No ha caido la promoción', 'No he ejecutado',
                         'No puedo señalizar', 'No tengo suficiente producto','Si', 'Si pero el precio no corresponde',
                         'Ejecutado', 'No Ejecutado', '% Ejecución']]

In [286]:
resumen_gos = data.groupby(['Respuesta Opc.Multiple/Texto Abierto_respuestas', 'Categoría'], as_index=False).count()
resumen_gos = resumen_gos.pivot_table(index=['Respuesta Opc.Multiple/Texto Abierto_respuestas'], columns='Categoría',
                                      values='Zona', fill_value=0)

In [287]:
sum_row = resumen_gos.sum()
sum_row = pd.DataFrame(sum_row).T

In [288]:
resumen_gos = resumen_gos.append(sum_row)

In [289]:
resumen_gos['% Detergentes'] = (resumen_gos['Detergentes'] / resumen_gos.iloc[6,0])
resumen_gos['% Suavizantes'] = (resumen_gos['Suavizantes'] / resumen_gos.iloc[6,1])
resumen_gos['% Lavatrastes'] = (resumen_gos['Lavatrastes'] / resumen_gos.iloc[6,2])
resumen_gos = resumen_gos.iloc[0:6,0:10]
resumen_gos.to_excel(writter, 'Resumen GOs')

In [290]:
active_row_names = ['No ha caido la promoción', 'No tengo suficiente producto', 
                    'No puedo señalizar']
active_filter = [row in active_row_names for row in resumen_gos.index]
inactive_filter = [not row for row in active_filter]
active = resumen_gos.loc[active_filter].sum()
active  = pd.DataFrame(active).transpose()
active.index=["NO, causal CT"] 
segundo = resumen_gos.append(active, ignore_index=False)

In [291]:
active_row_names = ['Si', 'Si pero el precio no corresponde']
active_filter = [row in active_row_names for row in segundo.index]
inactive_filter = [not row for row in active_filter]
active = resumen_gos.loc[active_filter].sum()
active  = pd.DataFrame(active).transpose()
active.index=["SI"] 
segundo = segundo.loc[inactive_filter].append(active, ignore_index=False)

In [292]:
segundo = segundo.sort_index(1)

In [293]:
segundo = segundo[['Detergentes', 'Suavizantes', 'Lavatrastes',  
                   '% Detergentes', '% Suavizantes', '% Lavatrastes']]

In [294]:
segundo.to_excel(writter, 'Resumen GOs', startrow=11)

In [295]:
no_ejecutado_region = data.groupby(['Región', 'Categoría','Respuesta Opc.Multiple/Texto Abierto_respuestas'],
                     as_index=False).count()

In [296]:
no_ejecutado_region = no_ejecutado_region[no_ejecutado_region['Respuesta Opc.Multiple/Texto Abierto_respuestas'] == 
                                          'No he ejecutado']

In [297]:
no_ejecutado_region = no_ejecutado_region.pivot_table(index='Región', columns='Categoría', values='Canal', fill_value=0)

In [298]:
no_ejecutado_region = no_ejecutado_region.join(no_ejecutado_region, lsuffix='_base')

In [299]:
no_ejecutado_region.to_excel(writter, 'Resumen GOs', startrow=22)

In [300]:
final = no_ejecutado_region.Detergentes[no_ejecutado_region['Detergentes']].count() + 23
worksheet = writter.sheets['Resumen GOs']
formato_gos_general = workbook.add_format()
formato_gos_general.set_font_color('white')
formato_gos_general.set_align('center')
formato_gos_general.set_bg_color('#222B35')
formato_gos_general.set_bold()
formato_gos_general.set_border()
worksheet.write('A1', 'Ejecución', formato_gos_general)
worksheet.write('A12', 'Ejecutando', formato_gos_general)
cabecera_resumen_gos(1, formato_gos_general)
cabecera_resumen_gos(12, formato_gos_general)
cabecera_resumen_gos(23, formato_gos_general)
worksheet.set_column('B:D', 21, None, {'hidden': True})
worksheet.set_column('E:G', 21, porcentaje)
worksheet.set_column('A:A', 34.5)
red_format = workbook.add_format({'bg_color':   '#FFC7CE',
                               'font_color': '#9C0006'})
green_format = workbook.add_format({'bg_color':   '#C6EFCE',
                               'font_color': '#006100'})
#worksheet.conditional_format('G24:K' + str(final), {'type':     'cell',
#                                    'criteria': 'greater than',
#                                    'value':    0,
#                                    'format':   red_format})
#worksheet.conditional_format('G24:K' + str(final), {'type':     'cell',
#                                    'criteria': 'less than',
#                                    'value':    1,
#                                    'format':   green_format})

In [301]:
reporte_mot = data.groupby(['Categoría', 'Cadena', 'Respuesta Opc.Multiple/Texto Abierto_respuestas'], as_index=False).count()

In [302]:
reporte_mot = reporte_mot.pivot_table(index=['Categoría', 'Cadena'], columns='Respuesta Opc.Multiple/Texto Abierto_respuestas',
                                      values='Zona', fill_value=0)

In [303]:
reporte_mot = reporte_mot.reset_index()

In [304]:
reporte_mot = reporte_mot[
    ['Categoría', 'Cadena', 'Si', 'Si pero el precio no corresponde', 'No ha caido la promoción', 'No puedo señalizar',
     'No tengo suficiente producto', 'No he ejecutado']]

In [305]:
reporte_mot['Ejecutado'] = (reporte_mot['Si'] + reporte_mot['Si pero el precio no corresponde'])

In [306]:
reporte_mot['No Ejecutado'] = (reporte_mot['No ha caido la promoción'] + reporte_mot['No he ejecutado'] + 
                               reporte_mot['No puedo señalizar'] + reporte_mot['No tengo suficiente producto'])

In [307]:
reporte_mot['% Ejecución'] = (reporte_mot['Ejecutado'] / (reporte_mot['Ejecutado'] + reporte_mot['No Ejecutado']))

In [308]:
reporte_mot['% No ha caido la promoción'] = (reporte_mot['No ha caido la promoción'] / reporte_mot['No Ejecutado'])

In [309]:
reporte_mot['% No puedo señalizar'] = (reporte_mot['No puedo señalizar'] / reporte_mot['No Ejecutado'])

In [310]:
reporte_mot['% No tengo suficiente producto'] = (reporte_mot['No tengo suficiente producto'] / reporte_mot['No Ejecutado'])

In [311]:
reporte_mot['% No he ejecutado'] = (reporte_mot['No he ejecutado'] / reporte_mot['No Ejecutado'])
reporte_mot = reporte_mot.fillna(0)

In [312]:
categorias_values = categorias.Categoría.unique()

In [313]:
formato_cabecera = workbook.add_format()
formato_cabecera.set_font_color('white')
formato_cabecera.set_align('center')
formato_cabecera.set_bg_color('#222B35')
formato_cabecera.set_bold()
formato_cabecera.set_text_wrap()
formato_cabecera.set_border()

In [314]:
formato_titulos = formato_cabecera

In [315]:
row = 2
for cat in categorias_values:
    reporte_mot[reporte_mot['Categoría'] == cat].to_excel(writter, 'Resumen MOT', startrow=row + 1, index=False, header=False)
    worksheet = writter.sheets['Resumen MOT']
    valores = reporte_mot.Cadena[reporte_mot['Categoría'] == cat].count()
    worksheet.merge_range('L' + str(row - 1) + ':O' + str(row - 1), 'Razones para no ejecutar', formato_cabecera)
    worksheet.merge_range('L' + str(row) + ':N' + str(row), 'Causal CT', formato_cabecera)
    worksheet.write('O' + str(row), 'Causal SDO', formato_cabecera)
    cabecera_resumen_mot(row + 1, formato_cabecera)
    worksheet.set_row(row, 33.75)
    worksheet.conditional_format('K' + str(row + 1) + ':K' + str(row + valores + 1), {'type': '3_color_scale'})
    row += (valores + 4)

In [316]:
for cat in categorias_values:
    categorias[categorias['Categoría'] == cat].to_excel(writter, cat, index=False)

In [317]:
worksheet = writter.sheets['Resumen MOT']

In [318]:
worksheet.set_column(0, 0, 18.29)
worksheet.set_column(1, 1, 21.29)
worksheet.set_column('C:J', None, None, {'hidden': True})
worksheet.set_column('K:O', 21, porcentaje)

0

In [319]:
def columnas_tiendas(tienda):
    tienda['Ejecutado'] = (tienda['Si'] + tienda['Si pero el precio no corresponde'])
    tienda['No Ejecutado'] = (tienda['No ha caido la promoción'] + tienda['No he ejecutado'] + 
                               tienda['No puedo señalizar'] + tienda['No tengo suficiente producto'])
    tienda['% Ejecución'] = (tienda['Ejecutado'] / (tienda['Ejecutado'] + tienda['No Ejecutado']))
    
def columnas_tiendas_sin_val(tienda):
    tienda['Ejecutado'] = (tienda['Si'] + tienda['Si pero el precio no corresponde'])
    tienda['No Ejecutado'] = (tienda['No ha caido la promoción'] + 
                               tienda['No puedo señalizar'] + tienda['No tengo suficiente producto'])
    tienda['% Ejecución'] = (tienda['Ejecutado'] / (tienda['Ejecutado'] + tienda['No Ejecutado']))

In [320]:
Soriana = order[order['Cadena'] == 'AC Soriana'].pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                                                   columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona',
                                                   fill_value=0).reset_index()

In [321]:
columnas_tiendas(Soriana)
Soriana.to_excel(writter, 'AC Soriana',index=False)

In [322]:
Chedraui = order[order['Cadena'] == 'AD Chedraui'].pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                                                    columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona',
                                                    fill_value=0).reset_index()

In [323]:
columnas_tiendas(Chedraui)
Chedraui.to_excel(writter, 'AD Chedraui', index=False)

In [324]:
Comercial_Mexicana = order[order['Cadena'] == 'AE Comercial Mexicana'].pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                                                              columns='Respuesta Opc.Multiple/Texto Abierto_respuestas',
                                                              values='Zona',
                                                              fill_value=0).reset_index()

In [325]:
columnas_tiendas_sin_val(Comercial_Mexicana)
Comercial_Mexicana.to_excel(writter,'AE Comercial Mexicana', index=False)

In [326]:
WM = order[(order['Cadena'] == 'AB Bodega Aurrera') | (order['Cadena'] == 'AA Supercenter') | (order[
                                                                                              'Cadena'] == 'Superama')].pivot_table(
    index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
    columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona',
    fill_value=0).reset_index()

In [327]:
columnas_tiendas(WM)
WM.to_excel(writter, 'WM', index=False)

In [328]:
por_tienda = data.groupby(['Canal', 'Categoría', 'Cadena', 'Nombre Tienda', 'Pregunta', 
                           'Respuesta Opc.Multiple/Texto Abierto_respuestas'], as_index=False).count()
cadenas = por_tienda.Cadena.unique()

for cadena in cadenas:
    por_tienda[por_tienda['Cadena'] == cadena][['Canal', 'Categoría', 'Cadena', 'Nombre Tienda', 'Pregunta', 
                                                'Respuesta Opc.Multiple/Texto Abierto_respuestas']].to_excel(writter, 
                                                                                                             'Data ' + cadena, 
                                                                                                             index=False)

In [329]:
workbook.close()
writter.save()

In [330]:
print('Ejecutado en: ' + str(time.time() - start_time) + ' segundos')

Ejecutado en: 26.608500003814697 segundos
